### 서포트벡터머신_수빈

In [12]:
import pandas as pd
final = pd.read_csv(r'final.csv', encoding = 'CP949')

final10 = final.copy()
final10 = final10.drop(['시도', '시군구', '행정번호', '년도'], axis=1)

from sklearn.preprocessing import RobustScaler
col = ['우울증환자수', '평균연령', '총인구수', '스트레스인지율', '고용률',
      '코로나확진자수', '1인당 정신건강예산(원)', '공공시설개수']

final20 = RobustScaler().fit_transform(final10[col])

final20 = pd.DataFrame(final20, columns = final10[col].columns)
final20['우울증지수'] = final10['우울증지수']
final20

,우울증환자수,평균연령,총인구수,스트레스인지율,고용률,코로나확진자수,1인당 정신건강예산(원),공공시설개수,우울증지수
0,2.358468,-0.075949,-0.090776,1.553571,-0.149153,-0.027095,-0.320131,-0.141176,9.693268
1,0.288788,-0.025316,-0.197140,0.500000,-0.149153,-0.027095,-0.320131,-0.164706,3.461523
2,-0.102448,-0.151899,0.204637,-0.982143,-0.149153,-0.027095,-0.320131,-0.035294,1.034939
3,0.151599,-0.354430,0.512841,1.035714,-0.149153,-0.027095,-0.320131,-0.070588,1.186486
4,0.795105,-0.430380,0.697005,0.696429,-0.149153,-0.027095,-0.320131,-0.105882,1.945387
...,...,...,...,...,...,...,...,...,...
1245,-0.514410,1.227848,-0.539560,0.357143,1.105085,5.100234,-0.031977,-0.588235,0.750504
1246,-0.413541,0.708861,-0.451330,-0.410714,0.854237,9.413112,-0.031977,-0.494118,1.314853
1247,-0.533754,1.645570,-0.522310,-0.428571,0.983051,5.038303,-0.031977,-0.647059,0.438992
1248,1.700355,-0.291139,1.233217,-0.089286,0.874576,87.528425,0.892078,1.670588,2.331426


In [3]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X = iris["data"][:, (2,3)] #꽃잎 길이/너비
y = (iris["target"] == 2).astype(np.float64)

svm_clf = Pipeline([("scaler", StandardScaler()),
                   ("linear_svc", LinearSVC(C=1, loss = "hinge")),])

svm_clf.fit(X, y)
svm_clf.predict([[5.5, 1.7]])

C:\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


array([1.])

In [6]:
from sklearn.datasets import make_moons
from sklearn.preprocessing import PolynomialFeatures

X, y = make_moons(n_samples=100, noise = 0.15)
polynomial_svm_clf = Pipeline([("poly_features", PolynomialFeatures(degree = 3)),
                              ("scaler", StandardScaler()),
                              ("svm_clf", LinearSVC(C=10, loss="hinge"))])
polynomial_svm_clf.fit(X, y)

C:\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('poly_features', PolynomialFeatures(degree=3)),
                ('scaler', StandardScaler()),
                ('svm_clf', LinearSVC(C=10, loss='hinge'))])

In [8]:
from sklearn.svm import SVC
poly_kernel_svm_clf = Pipeline([("scaler", StandardScaler()),
                              ("svm_clf", SVC(kernel = "poly",
                                             degree = 3,
                                             coef0 = 1,
                                             C = 5))])
poly_kernel_svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=5, coef0=1, kernel='poly'))])

In [54]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

X = final20[['평균연령', '총인구수', '스트레스인지율', '고용률', '코로나확진자수', '1인당 정신건강예산(원)', '공공시설개수']]
y = final20['우울증지수']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

svm_regressor = SVR(kernel='rbf', C=10, gamma=1)

svm_regressor.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = svm_regressor.predict(X_test)

# 평가 지표 출력
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


Mean Squared Error: 0.7308920491974527


In [ ]:
result_df = pd.DataFrame({'실제값': y_test, '예측값': y_pred})

# 결과 출력
result_df

In [52]:
# 그리드서치로 최적의 gamma값 찾기 -> 1
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.model_selection import train_test_split
import pandas as pd

# 데이터 준비 (final20 데이터셋 사용)
# X, y를 적절히 설정하여 데이터를 준비하세요.
X = final20[['평균연령', '총인구수', '스트레스인지율', '고용률', '코로나확진자수', '1인당 정신건강예산(원)', '공공시설개수']]
y = final20['우울증지수']

# 훈련 세트와 테스트 세트로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# SVR 모델 정의
svr_model = SVR(kernel='rbf')

# 탐색할 gamma 값의 후보 리스트 정의
gamma_candidates = [0.001, 0.01, 0.1, 1, 10, 100]

# 그리드 서치를 위한 매개변수 그리드 생성
param_grid = {'gamma': gamma_candidates}

# 5-폴드 교차 검증을 사용하여 그리드 서치 수행
grid_search = GridSearchCV(svr_model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# 최적의 파라미터와 성능 출력
print("최적의 gamma:", grid_search.best_params_)
print("최적의 평균 제곱 오차 (MSE):", -grid_search.best_score_)

# 각 gamma 값에 대한 성능 출력
results = pd.DataFrame(grid_search.cv_results_)
print(results[['params', 'mean_test_score', 'std_test_score']])

# 테스트 세트에서 모델 평가
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred)
print(f"테스트 세트 MSE: {mse_test}")

최적의 gamma: {'gamma': 1}
최적의 평균 제곱 오차 (MSE): 1.2469586232059757
             params  mean_test_score  std_test_score
0  {'gamma': 0.001}        -1.383761        0.241757
1   {'gamma': 0.01}        -1.343133        0.248647
2    {'gamma': 0.1}        -1.253712        0.252778
3      {'gamma': 1}        -1.246959        0.216296
4     {'gamma': 10}        -1.426810        0.219736
5    {'gamma': 100}        -1.593431        0.255108
테스트 세트 MSE: 0.8477339965948358


In [51]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.model_selection import train_test_split
import pandas as pd

# 데이터 준비 (final20 데이터셋 사용)
X = final20[['평균연령', '총인구수', '스트레스인지율', '고용률', '코로나확진자수', '1인당 정신건강예산(원)', '공공시설개수']]
y = final20['우울증지수']

# 훈련 세트와 테스트 세트로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# SVR 모델 정의
svr_model = SVR(kernel='rbf')

# 탐색할 C 값의 후보 리스트 정의
C_candidates = [0.001, 0.01, 0.1, 1, 10, 100]

# 그리드 서치를 위한 매개변수 그리드 생성
param_grid = {'C': C_candidates}

# 5-폴드 교차 검증을 사용하여 그리드 서치 수행
grid_search = GridSearchCV(svr_model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# 최적의 파라미터와 성능 출력
print("최적의 C:", grid_search.best_params_)
print("최적의 평균 제곱 오차 (MSE):", -grid_search.best_score_)

# 각 C 값에 대한 성능 출력
results = pd.DataFrame(grid_search.cv_results_)
print(results[['params', 'mean_test_score', 'std_test_score']])

# 테스트 세트에서 모델 평가
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred)
print(f"테스트 세트 MSE: {mse_test}")

최적의 C: {'C': 100}
최적의 평균 제곱 오차 (MSE): 1.295956707463933
         params  mean_test_score  std_test_score
0  {'C': 0.001}        -1.585824        0.258518
1   {'C': 0.01}        -1.539667        0.259696
2    {'C': 0.1}        -1.467854        0.250456
3      {'C': 1}        -1.360499        0.239940
4     {'C': 10}        -1.325776        0.246007
5    {'C': 100}        -1.295957        0.257454
테스트 세트 MSE: 0.9516951989965897


In [66]:
# 최적의 k값 구하기
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline

# 예제 데이터셋과 변수
X = final20[['평균연령', '총인구수', '스트레스인지율', '고용률', '코로나확진자수', '1인당 정신건강예산(원)', '공공시설개수']]
y = final20['우울증지수']

# SVM 모델 정의
svm_regressor = SVR(kernel='rbf', C=100, gamma=1)

# SelectKBest를 사용한 변수 선택
k_best = SelectKBest(f_regression)

# Pipeline 정의
pipeline = Pipeline([('selectkbest', k_best), ('svr', svm_regressor)])

# 탐색할 k 값의 범위 설정
param_grid = {'selectkbest__k': list(range(1, 8))}

# GridSearchCV를 사용하여 최적의 k 값 찾기
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X, y)

# 최적의 k 값 출력
best_k = grid_search.best_params_['selectkbest__k']
print(f"Best k value: {best_k}")

Best k value: 3


In [67]:
# 종속변수와 가장 상관관계가 강한 3개의 특성 선택됨
# 상관관계가 높다고 해서 항상 변수가 유용하다고는 할 수 없음
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 예제 데이터셋과 변수
X = final20[['평균연령', '총인구수', '스트레스인지율', '고용률', '코로나확진자수', '1인당 정신건강예산(원)', '공공시설개수']]
y = final20['우울증지수']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# SelectKBest를 사용한 변수 선택
k_best = SelectKBest(f_regression, k=3)
X_train_kbest = k_best.fit_transform(X_train, y_train)
selected_features_mask = k_best.get_support()
selected_features = X.columns[selected_features_mask]

print("Selected features:", selected_features)

Selected features: Index(['평균연령', '총인구수', '고용률'], dtype='object')


In [68]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.model_selection import train_test_split
import pandas as pd

# 데이터 준비 (final20 데이터셋 사용)
# X, y를 적절히 설정하여 데이터를 준비하세요.
X = final20[['평균연령', '총인구수', '고용률']]
y = final20['우울증지수']

# 훈련 세트와 테스트 세트로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# SVR 모델 정의
svr_model = SVR(kernel='rbf')

# 탐색할 gamma 값의 후보 리스트 정의
gamma_candidates = [0.001, 0.01, 0.1, 1, 10, 100]

# 그리드 서치를 위한 매개변수 그리드 생성
param_grid = {'gamma': gamma_candidates}

# 5-폴드 교차 검증을 사용하여 그리드 서치 수행
grid_search = GridSearchCV(svr_model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# 최적의 파라미터와 성능 출력
print("최적의 gamma:", grid_search.best_params_)
print("최적의 평균 제곱 오차 (MSE):", -grid_search.best_score_)

# 각 gamma 값에 대한 성능 출력
results = pd.DataFrame(grid_search.cv_results_)
print(results[['params', 'mean_test_score', 'std_test_score']])

# 테스트 세트에서 모델 평가
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred)
print(f"테스트 세트 MSE: {mse_test}")

최적의 gamma: {'gamma': 10}
최적의 평균 제곱 오차 (MSE): 1.0810018905791954
             params  mean_test_score  std_test_score
0  {'gamma': 0.001}        -1.427871        0.240485
1   {'gamma': 0.01}        -1.375348        0.235176
2    {'gamma': 0.1}        -1.276786        0.235559
3      {'gamma': 1}        -1.150112        0.222005
4     {'gamma': 10}        -1.081002        0.195728
5    {'gamma': 100}        -1.266935        0.221555
테스트 세트 MSE: 0.7922429380482555


In [69]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.model_selection import train_test_split
import pandas as pd

# 데이터 준비 (final20 데이터셋 사용)
X = final20[['평균연령', '총인구수', '고용률']]
y = final20['우울증지수']

# 훈련 세트와 테스트 세트로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# SVR 모델 정의
svr_model = SVR(kernel='rbf')

# 탐색할 C 값의 후보 리스트 정의
C_candidates = [0.001, 0.01, 0.1, 1, 10, 100]

# 그리드 서치를 위한 매개변수 그리드 생성
param_grid = {'C': C_candidates}

# 5-폴드 교차 검증을 사용하여 그리드 서치 수행
grid_search = GridSearchCV(svr_model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# 최적의 파라미터와 성능 출력
print("최적의 C:", grid_search.best_params_)
print("최적의 평균 제곱 오차 (MSE):", -grid_search.best_score_)

# 각 C 값에 대한 성능 출력
results = pd.DataFrame(grid_search.cv_results_)
print(results[['params', 'mean_test_score', 'std_test_score']])

# 테스트 세트에서 모델 평가
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred)
print(f"테스트 세트 MSE: {mse_test}")

최적의 C: {'C': 100}
최적의 평균 제곱 오차 (MSE): 1.1011007870495408
         params  mean_test_score  std_test_score
0  {'C': 0.001}        -1.545796        0.253549
1   {'C': 0.01}        -1.389809        0.231868
2    {'C': 0.1}        -1.279217        0.221891
3      {'C': 1}        -1.164482        0.230139
4     {'C': 10}        -1.113444        0.214021
5    {'C': 100}        -1.101101        0.202934
테스트 세트 MSE: 0.9556872147724419


In [82]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

X = final20[['평균연령', '총인구수', '고용률']]
y = final20['우울증지수']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

svm_regressor = SVR(kernel='rbf', C=100, gamma=10)

svm_regressor.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = svm_regressor.predict(X_test)

# 평가 지표 출력
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")ㅊ

Mean Squared Error: 1.0992650849955083


In [81]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

X = final20[['평균연령', '총인구수', '고용률']]
y = final20['우울증지수']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

svm_regressor = SVR(kernel='rbf', C=100, gamma=10)

svm_regressor.fit(X_train, y_train)

# 훈련 세트에서의 예측
y_train_pred = svm_regressor.predict(X_train)

# 검증 세트에서의 예측
y_val_pred = svm_regressor.predict(X_test)

# 훈련 세트에서의 평가 지표 계산 (MSE 사용)
mse_train = mean_squared_error(y_train, y_train_pred)
print(f"Training Set Mean Squared Error: {mse_train}")

# 검증 세트에서의 평가 지표 계산 (MSE 사용)
mse_val = mean_squared_error(y_test, y_val_pred)
print(f"Validation Set Mean Squared Error: {mse_val}")

Training Set Mean Squared Error: 0.339012607310212
Validation Set Mean Squared Error: 1.0992650849955083


In [85]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# 데이터 생성 또는 불러오기
X = final20[['평균연령', '총인구수', '스트레스인지율', '고용률', '코로나확진자수', '1인당 정신건강예산(원)', '공공시설개수']]
y = final20['우울증지수']

# 데이터를 훈련 세트와 검증 세트로 나누기
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)

svm_regressor = SVR(kernel='rbf', C=100, gamma=10)

svm_regressor.fit(X_train, y_train)

# 훈련 세트에서의 예측
y_train_pred = svm_regressor.predict(X_train)

# 검증 세트에서의 예측
y_val_pred = svm_regressor.predict(X_val)

# 훈련 세트에서의 평가 지표 계산 (MSE 사용)
mse_train = mean_squared_error(y_train, y_train_pred)
print(f"Training Set Mean Squared Error: {mse_train}")

# 검증 세트에서의 평가 지표 계산 (MSE 사용)
mse_val = mean_squared_error(y_test, y_val_pred)
print(f"Validation Set Mean Squared Error: {mse_val}")

Training Set Mean Squared Error: 0.009424064293490797
Validation Set Mean Squared Error: 0.9439606231157293


In [86]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor

# 예제 데이터셋과 변수
X = final20[['평균연령', '총인구수', '스트레스인지율', '고용률', '코로나확진자수', '1인당 정신건강예산(원)', '공공시설개수']]
y = final20['우울증지수']

# 모델 정의
model = RandomForestRegressor()

# RFE를 사용하여 변수 선택 (여기서는 3개 선택)
rfe = RFE(model, n_features_to_select=3)
X_rfe = rfe.fit_transform(X, y)

# 선택된 변수 출력
selected_features = X.columns[rfe.support_]
print("Selected features:", selected_features)

Selected features: Index(['총인구수', '고용률', '공공시설개수'], dtype='object')


In [87]:
from sklearn.ensemble import RandomForestRegressor

# 예제 데이터셋과 변수
X = final20[['평균연령', '총인구수', '스트레스인지율', '고용률', '코로나확진자수', '1인당 정신건강예산(원)', '공공시설개수']]
y = final20['우울증지수']

# 모델 정의
model = RandomForestRegressor()

# 모델을 훈련하여 변수의 중요도 측정
model.fit(X, y)

# 변수의 중요도 출력
feature_importance = model.feature_importances_
print("Feature importance:", feature_importance)

# 중요도가 높은 상위 3개 변수 선택
selected_features = X.columns[np.argsort(feature_importance)[::-1][:3]]
print("Selected features:", selected_features)

Feature importance: [0.10951809 0.40311788 0.09003799 0.14506851 0.05392204 0.06246228
 0.1358732 ]
Selected features: Index(['총인구수', '고용률', '공공시설개수'], dtype='object')


In [91]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

X = final20[['총인구수', '고용률', '공공시설개수']]
y = final20['우울증지수']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

svm_regressor = SVR(kernel='rbf', C=100, gamma=10)

svm_regressor.fit(X_train, y_train)

# 훈련 세트에서의 예측
y_train_pred = svm_regressor.predict(X_train)

# 검증 세트에서의 예측
y_val_pred = svm_regressor.predict(X_test)

# 훈련 세트에서의 평가 지표 계산 (MSE 사용)
mse_train = mean_squared_error(y_train, y_train_pred)
print(f"Training Set Mean Squared Error: {mse_train}")

# 검증 세트에서의 평가 지표 계산 (MSE 사용)
mse_val = mean_squared_error(y_test, y_val_pred)
print(f"Validation Set Mean Squared Error: {mse_val}")

Training Set Mean Squared Error: 0.31504886193004156
Validation Set Mean Squared Error: 1.141911215193316


In [104]:
X = final20[['평균연령', '총인구수', '스트레스인지율', '고용률', '코로나확진자수', '1인당 정신건강예산(원)', '공공시설개수']]
y = final20['우울증지수']



X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    random_state = 42)

svm_regressor = SVR(kernel='rbf', C=100, gamma=10)

svm_regressor.fit(X_train, y_train)

y_train_pred = svm_regressor.predict(X_train)

# 검증 세트에서의 예측
y_test_pred = svm_regressor.predict(X_test)


mse = mean_squared_error(y_test, y_test_pred)
print(mse)



from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X, y,
                         scoring='neg_mean_squared_error', cv=7)
tree_rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("-------------------")
    print("rmse 점수:", scores)
    print("평균 rmse:", scores.mean())
    print("rmse 표준편차:", scores.std())
    
display_scores(tree_rmse_scores)

0.9439606231157293
-------------------
rmse 점수: [0.91396361 0.88436423 0.74839474 0.6582817  0.82649714 0.79928733
 0.91037056]
평균 rmse: 0.8201656166953759
rmse 표준편차: 0.08692353576491577


X = final20[['평균연령', '총인구수', '스트레스인지율', '고용률',   
            '코로나확진자수', '1인당 정신건강예산(원)', '공공시설개수']]  
y = final20['우울증지수']

Training Set Mean Squared Error: 0.009424064293490797  
Validation Set Mean Squared Error: 0.9439606231157293  

일때의 결과가 SVR 모델의 가장 최적의 성능이지만 조금 과대적합의 경향이 있음  

다항 회귀의 결과가  
Train MSE: 1.0193099152540395  
Validation MSE: 0.8762869715877589  
로 나오므로 성능이 더 좋음